Join with sales tables and get sales information of each users by date.

In [0]:
%sql
create table demo_catalog.demo_schema.member_sales
(
  member_id string,
  date date,
  transaction_count int,
  total decimal(10,2)
)

In [0]:
%sql
create or replace temporary view member_sale_view as(
select 
  sale.member_id,
  date(sale.date) as date,
  count(*) as transaction_count,
  COALESCE(ROUND(SUM(sale.total), 2), 0) as total
from demo_catalog.demo_schema.sales_transaction as sale
  left join demo_catalog.demo_schema.members as member on sale.member_id = member.member_id
group by 
  sale.member_id,
  sale.date
)

In [0]:
%sql
merge into demo_catalog.demo_schema.member_sales as member_sales
using member_sale_view
on member_sales.member_id = member_sale_view.member_id 
and member_sales.date = member_sale_view.date 
when matched then
  update set
  transaction_count = member_sale_view.transaction_count,
  total = member_sale_view.total
when not matched 
  then insert(
    member_id,
    date,
    transaction_count,
    total
  )
  values(
    member_sale_view.member_id,
    member_sale_view.date,
    member_sale_view.transaction_count,
    member_sale_view.total
  )

In [0]:
#truncate and write data only

driver = "org.mariadb.jdbc.Driver"
table = "demo_analytic_database.member_sales"
user = dbutils.secrets.get("jdbc", "username")
password = dbutils.secrets.get("jdbc", "password")
url = "jdbc:mysql://host_address/demo_analytic_database?useLegacyDatetimeCode=false"

df = spark.table("demo_catalog.demo_schema.member_sales")

df.write.format("jdbc")\
    .option("driver", driver)\
    .option("url", url)\
    .option("dbtable", table)\
    .option("truncate", "true")\
    .option("user", user)\
    .option("password", password)\
    .mode("overwrite")\
    .save()

In [0]:
#